# Introduction

In this document, we are concerned with a (seemingly) simple mathematical problem: Given a positive number $x$, what is the power of two nearest to $x$?

A common solution to this problem is implemented below. This is used in recent [research](https://arxiv.org/pdf/2210.03671.pdf) in quantized deep learning and the [qkeras](https://github.com/google/qkeras) library. 

In [6]:
import tensorflow as tf
from tensorflow.keras import backend as K

def get_nearest_power_of_two(x):

  return K.pow(2.0, tf.math.round(K.log(x + K.epsilon()) / K.log(2.0)))

# A Closer Look

To see how well this works, let's take a close look at what happens near the value 3. Clearly all values slightly less then 3 should go to 2, and all values slightly above 3 should go to 4.

In [7]:
for x in [2.9, 3.0, 3.1]:
  print(f'{x = }')
  print(f'{get_nearest_power_of_two(x).numpy() = }')
  print()

x = 2.9
get_nearest_power_of_two(x).numpy() = 4.0

x = 3.0
get_nearest_power_of_two(x).numpy() = 4.0

x = 3.1
get_nearest_power_of_two(x).numpy() = 4.0



It appears that this formula is biased towards higher powers of two. 

# A Better Idea

We've fixed this up with a new function below. 

In [18]:
def get_nearest_power_of_two_improved(x):

  return K.pow(2.0, tf.math.round(K.log(x/1.5 + K.epsilon()) / K.log(2.0) + 0.5))

for x in [2.9, 3.0, 3.1]:
  print(f'{x = }')
  print(f'{get_nearest_power_of_two_improved(x).numpy() = }')
  print()

x = 2.9
get_nearest_power_of_two_improved(x).numpy() = 2.0

x = 3.0
get_nearest_power_of_two_improved(x).numpy() = 4.0

x = 3.1
get_nearest_power_of_two_improved(x).numpy() = 4.0



This looks better!

# Tests

To test this function, let's use a slower but more surefire way of determining the power of two nearest to a number: iteration. We can benchmark against this to see if this formula works well. 

In [44]:
import numpy as np
from tqdm import tqdm

def iterative_nearest_power_of_two(x):

  max_exp_for_iteration = int(np.ceil(np.log(x) / np.log(2.0)))

  distances = []

  for i in range(0, max_exp_for_iteration + 3):
    distances.append(np.abs(x - np.power(2, i)))

  min_index = np.argmin(distances)

  return 2 ** min_index

def test_po2_formula(x):

  iterative_nearest = iterative_nearest_power_of_two(x)
  formula_nearest = get_nearest_power_of_two_improved(tf.cast(x, tf.float32)).numpy()

  assert np.isclose(abs(x - iterative_nearest), abs(x - formula_nearest))

for x in tqdm(list(np.random.uniform(low=1, high=1000, size=(1000,)))):
  test_po2_formula(x)
print()
print('Tests passed!')


100%|██████████| 1000/1000 [00:03<00:00, 296.58it/s]


Tests passed!
